In [3]:
from sentence_transformers import SentenceTransformer
import markdown
import os

# 마크다운 파일 로드
with open("finetune-markdown.md", "r") as file:
    markdown_text = file.read()

text = markdown.markdown(markdown_text)		# 마크다운을 일반 텍스트로 변환
sections = text.split("\n\n")  				# 헤더로 구분된 섹션을 텍스트를 섹션으로 분할, 마크다운 구조에 따라 분할 기준 조정
print(sections)

['<p>① 회사는 채용이 확정된 자와 서면으로 근로계약을 체결하고, 해당자에게 근로계약서 사본 1부를 내어 준다.\n② 회사는 근로계약 체결 시 사원에게 임금, 소정근로시간, 휴게시간, 휴일, 연차 유급휴가, 취업의 장소와 종사하여야 할 업무에 관한 사항, 근로기준법 제93조제1호부터 제12호까지의 규정(취업규칙의 작성 ․ 신고사항)에서 정한 사항, 근로기준법 제10장의 기숙사에 관한 사항(기숙사가 있는 경우에 한함)을 명확히 제시한다.\n③ 회사는 제2항의 내용 중 임금의 구성항목 ․ 계산방법 ․ 지급방법, 소정근로시간, 휴게시간, 휴일, 연차유급휴가에 관한 사항, 취업의 장소와 종사하여야 할 업무에 관한 사항을 서면으로 명확히 제시하여 교부한다. 또한 기간제 근로자인 경우 근로계약기간도 함께 명시한다.\n④ 회사는 근로계약 체결 시 제2항 및 제3항의 사항이 적시된 취업규칙을 제시하거나 교부함으로써 제2항의 명시 및 제3항의 서면명시 및 교부의무를 대신할 수 있다.</p>']


In [4]:
# 사전 학습된 임베딩 모델 로드
model = SentenceTransformer('all-MiniLM-L6-v2')

# 각 섹션 벡터 임베딩
embeddings = model.encode(sections)

# 이제 `embeddings`에는 마크다운 문서의 각 섹션에 대한 벡터 표현이 포함됩니다.

In [7]:
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

def find_relevant_section(question, sections, embeddings):
    question_embedding = model.encode([question])
    similarities = cosine_similarity(question_embedding, embeddings)
    most_relevant_idx = np.argmax(similarities)
    return sections[most_relevant_idx]

# Load a QA model
qa_model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased-distilled-squad")
qa_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-distilled-squad")

def answer_question(question, context, max_length=512):
    # 질문과 맥락을 토큰화하고 필요한 경우 잘라냅니다.
    inputs = qa_tokenizer.encode_plus(question, context, add_special_tokens=True, max_length=max_length, truncation=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    # Forward pass through the model
    outputs = qa_model(**inputs)
    answer_start_scores, answer_end_scores = outputs.start_logits, outputs.end_logits

    # '시작' 및 '끝' 점수가 가장 높은 토큰을 찾습니다.
    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1

    # 토큰을 문자열로 변환
    answer = qa_tokenizer.convert_tokens_to_string(qa_tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    return answer


# Using the model
question = "What is the main topic?"
relevant_section = find_relevant_section(question, sections, embeddings)
print(answer_question(question, relevant_section))


[SEP]
